In [ ]:
!pip -q install open-clip-torch ultralytics pillow scikit-learn

import os, pandas as pd, torch
BASE = "/content/drive/MyDrive/abid-mvp"
PROC = f"{BASE}/data/processed"
RAW  = f"{BASE}/data/raw/bin-images"

TRAIN = pd.read_csv(f"{PROC}/train.csv")
VAL   = pd.read_csv(f"{PROC}/val.csv")

ASIN_COL = "asin"  # from catalog_expanded
print("Loaded:", TRAIN.shape, VAL.shape, "| device:", "cuda" if torch.cuda.is_available() else "cpu")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00
Loaded: (66, 3) (31, 3) | device: cpu


In [ ]:
import numpy as np
from PIL import Image
import open_clip

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, _, preprocess = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")
clip_model = clip_model.to(DEVICE).eval()

def embed_img_path(p):
    if isinstance(p, str):
        im = Image.open(p).convert("RGB")
    elif isinstance(p, Image.Image):
        im = p.convert("RGB") # Already a PIL Image, just convert format
    else:
        raise TypeError("Input must be a file path (str) or a PIL Image object")

    with torch.no_grad():
        t = preprocess(im).unsqueeze(0).to(DEVICE)
        e = clip_model.encode_image(t).cpu().numpy()
    return e

# pick up to K images per ASIN as templates
K = 3
templates = {asin: g["filename"].dropna().head(K).tolist()
             for asin, g in TRAIN.groupby(ASIN_COL)}

asin_emb = {}
for asin, files in templates.items():
    vecs = []
    for fn in files:
        p = os.path.join(RAW, fn)
        if os.path.exists(p):
            vecs.append(embed_img_path(p))
    if vecs:
        asin_emb[asin] = np.vstack(vecs).mean(axis=0, keepdims=True)

len(asin_emb)

/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


30

In [ ]:
from ultralytics import YOLO
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

yolo = YOLO("yolov8n.pt")  # small & fast
SCORE_THR = 0.28  # tune later

def count_asin_in_image(img_path, asin):
    if asin not in asin_emb: return 0, []
    im = Image.open(img_path).convert("RGB")
    res = yolo.predict(source=np.array(im), verbose=False, conf=0.1)[0]
    crops = []
    for (x1,y1,x2,y2) in res.boxes.xyxy.cpu().numpy().astype(int):
        x1,y1 = max(0,x1), max(0,y1)
        x2,y2 = min(im.width,x2), min(im.height,y2)
        if x2-x1<10 or y2-y1<10: continue
        crops.append(im.crop((x1,y1,x2,y2)))
    if not crops: return 0, []
    embs = np.vstack([embed_img_path(c) for c in crops])
    sims = cosine_similarity(embs, asin_emb[asin]).ravel()
    idx = [i for i,s in enumerate(sims) if s >= SCORE_THR]
    return len(idx), sorted([(int(i), float(sims[i])) for i in idx], key=lambda x: -x[1])

In [ ]:
ok = 0
checked = 0
for _, r in VAL.head(50).iterrows():  # small probe
    fp = os.path.join(RAW, r["filename"])
    if not os.path.exists(fp):
        continue
    cnt, _ = count_asin_in_image(fp, r[ASIN_COL])
    gt = int(r.get("quantity", 1))
    ok += int(cnt >= gt)  # crude pass criterion
    checked += 1

print(f"Bin-level rough pass: {ok}/{checked}")

# save embeddings for the Streamlit app
import pickle, pathlib
ART = f"{BASE}/models"
pathlib.Path(ART).mkdir(parents=True, exist_ok=True)
with open(f"{ART}/asin_emb.pkl","wb") as f:
    pickle.dump(asin_emb, f)
print("Saved:", f"{ART}/asin_emb.pkl")


Bin-level rough pass: 13/31
Saved: /content/drive/MyDrive/abid-mvp/models/asin_emb.pkl


In [ ]:
# CPU-mode: build/save asin_emb here, then use infer.count_asin_in_image
import os, sys, numpy as np, pandas as pd, pickle, torch, open_clip
from PIL import Image
import os, importlib, sys
sys.path.append("/content/drive/MyDrive/abid-mvp/src")

os.environ["CLIP_MODEL"] = "ViT-B-32"
os.environ["CLIP_TAG"]   = "openai"

import infer
importlib.reload(infer)

from infer import count_asin_in_image  # re-import after reload

# --- Paths ---
BASE = "/content/drive/MyDrive/abid-mvp"
RAW  = f"{BASE}/data/raw/bin-images"
PROC = f"{BASE}/data/processed"
MODELS = f"{BASE}/models"
os.makedirs(MODELS, exist_ok=True)

# --- Import the CPU infer (grid crops, ViT-B/32) ---
SRC = f"{BASE}/src"
if SRC not in sys.path: sys.path.insert(0, SRC)
from infer import count_asin_in_image  # only this is needed now

# --- CLIP (ViT-B/32) for building template bank ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, _, preprocess = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")
clip_model = clip_model.to(DEVICE).eval()
tokenizer = open_clip.get_tokenizer("ViT-B-32")

def embed_img(path_or_pil):
    img = Image.open(path_or_pil).convert("RGB") if isinstance(path_or_pil, str) else path_or_pil.convert("RGB")
    with torch.no_grad():
        t = preprocess(img).unsqueeze(0).to(DEVICE)
        return clip_model.encode_image(t).cpu().numpy()

def embed_text(text):
    with torch.no_grad():
        tok = tokenizer([text]).to(DEVICE)
        return clip_model.encode_text(tok).cpu().numpy()

# --- Load dataframes ---
TRAIN = pd.read_csv(f"{PROC}/train.csv")
VAL   = pd.read_csv(f"{PROC}/val.csv")
ASIN_COL = "asin"; FILE_COL = "filename"

# --- Optional catalog for text fallback ---
catalog_path = os.path.join(PROC, "catalog_expanded.csv")
CAT = pd.read_csv(catalog_path) if os.path.exists(catalog_path) else pd.DataFrame()
CAT[ASIN_COL] = CAT.get(ASIN_COL, pd.Series(dtype=str)).astype(str)
TRAIN[ASIN_COL] = TRAIN[ASIN_COL].astype(str)

# pick reasonable text columns for fallback
text_cols = [c for c in CAT.columns if any(k in c.lower() for k in ["title","name","desc","product","label","brand"])]

# --- Build asin_emb (K=5 images per ASIN, else text fallback) ---
K = 5
asin_emb = {}
for asin, g in TRAIN.groupby(ASIN_COL):
    vecs = []
    for fn in g[FILE_COL].dropna().head(K).tolist():
        fp = os.path.join(RAW, fn)
        if os.path.exists(fp):
            try: vecs.append(embed_img(fp))
            except: pass
    if vecs:
        asin_emb[asin] = np.vstack(vecs).mean(axis=0, keepdims=True)

# text fallback for missing
for asin in TRAIN[ASIN_COL].unique():
    if asin in asin_emb:
        continue
    prompt = None
    if not CAT.empty and asin in set(CAT[ASIN_COL]):
        row = CAT.loc[CAT[ASIN_COL]==asin].iloc[0]
        for c in text_cols:
            s = str(row.get(c, "")).strip()
            if s and s.lower() not in ["nan","none","null"]:
                prompt = s; break
    if not prompt:
        prompt = f"Retail packaged product {asin}"
    try:
        asin_emb[asin] = embed_text(prompt)
    except:
        asin_emb[asin] = embed_text(f"Product {asin}")

# --- Save for reuse ---
with open(os.path.join(MODELS, "asin_emb.pkl"), "wb") as f:
    pickle.dump(asin_emb, f)
print(f"✅ asin_emb saved → {MODELS}/asin_emb.pkl | coverage {len(asin_emb)}/{TRAIN[ASIN_COL].nunique()}")

# --- Quick smoke test on a real row from VAL that exists on disk ---
test_row = None
for _, r in VAL.iterrows():
    fp = os.path.join(RAW, r[FILE_COL])
    if os.path.exists(fp) and str(r[ASIN_COL]) in asin_emb:
        test_row = r; break

if test_row is not None:
    fp = os.path.join(RAW, test_row[FILE_COL])
    asin = str(test_row[ASIN_COL])
    cnt, hits, boxes = count_asin_in_image(fp, asin, asin_emb, score_thresh=0.20)  # CPU/grid uses thr≈0.20
    print("🔎 Test:", os.path.basename(fp), "| ASIN:", asin, "| cnt:", cnt, "| top hits:", hits[:3])
else:
    print("⚠️ No valid test row found (file missing or missing embedding).")


/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


✅ asin_emb saved → /content/drive/MyDrive/abid-mvp/models/asin_emb.pkl | coverage 30/30


/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


🔎 Test: 00642.jpg | ASIN: B01BNJL3KC | cnt: 9 | top hits: [(0, 0.8649296760559082), (2, 0.8550341129302979), (1, 0.8527300953865051)]


In [ ]:
print(asin_emb)

{'B000KI8HKC': array([[   0.033061,   -0.093907,   -0.086728,     0.21967,     0.29848,     0.43791,     0.27244,     0.68414,    -0.33896,  -0.0032021,     0.57846,    0.035283,      0.5357,   -0.014711,    -0.13103,    -0.35545,    -0.17258,   -0.047887,     0.41828,  -0.0099978,    -0.26522,    -0.37692,     0.29711,
            0.10041,     0.07941,     0.28648,    -0.32936,    -0.20928,    0.018735,     0.14965,     0.30216,    0.075631,   0.0035174,     -0.1037,    -0.19308,     0.36935,     0.05707,    0.063479,   0.0036962,    -0.15593,     0.12757,    0.086356,    -0.37832,    -0.59509,       0.139,    -0.55069,
            0.22833,     0.14099,    -0.31032,     0.14592,     -0.1002,    -0.06459,    -0.12242,    0.062794,    0.033837,   -0.028456,    -0.48304,     0.11727,    -0.11063,     0.40437,    -0.28938,   -0.036472,     0.32896,     0.23217,     -0.1299,     0.23746,   -0.095008,     -0.1801,    -0.10226,
           -0.24327,     0.20358,     0.01973,    -0.15148,    -

In [ ]:
# A) load splits
import os, pandas as pd, torch
BASE = "/content/drive/MyDrive/abid-mvp"
PROC = f"{BASE}/data/processed"
RAW  = f"{BASE}/data/raw/bin-images"
TRAIN = pd.read_csv(f"{PROC}/train.csv")
VAL   = pd.read_csv(f"{PROC}/val.csv")
ASIN_COL = "asin"
print(TRAIN.shape, VAL.shape, "device:", "cuda" if torch.cuda.is_available() else "cpu")


(66, 3) (31, 3) device: cpu


In [ ]:
# B) CLIP templates (centroids)
!pip -q install open-clip-torch ultralytics pillow scikit-learn
import numpy as np
from PIL import Image
import open_clip

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, _, preprocess = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")
clip_model = clip_model.to(DEVICE).eval()

def embed_img(p):
    if isinstance(p, str):
        im = Image.open(p).convert("RGB")
    elif isinstance(p, Image.Image):
        im = p.convert("RGB") # Already a PIL Image, just convert format
    else:
        raise TypeError("Input must be a file path (str) or a PIL Image object")

    with torch.no_grad():
        t = preprocess(im).unsqueeze(0).to(DEVICE)
        return clip_model.encode_image(t).cpu().numpy()

K = 3
templates = {a: g["filename"].dropna().head(K).tolist() for a,g in TRAIN.groupby(ASIN_COL)}
asin_emb = {}
for a, files in templates.items():
    vecs = []
    for fn in files:
        fp = os.path.join(RAW, fn)
        if os.path.exists(fp):
            try: vecs.append(embed_img(fp))
            except: pass
    if vecs:
        asin_emb[a] = np.vstack(vecs).mean(axis=0, keepdims=True)

len(asin_emb)

/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


30

In [ ]:
# === Build asin_emb (ViT-B/32 + Text Fallback) ===
!pip -q install open-clip-torch pillow scikit-learn

import os, numpy as np, pandas as pd, torch, open_clip
from PIL import Image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, _, preprocess = open_clip.create_model_and_transforms(
    "ViT-B-32", pretrained="openai"
)
clip_model = clip_model.to(DEVICE).eval()

def embed_img(img_or_path):
    if isinstance(img_or_path, str):
        img = Image.open(img_or_path).convert("RGB")
    else:
        img = img_or_path.convert("RGB")
    with torch.no_grad():
        t = preprocess(img).unsqueeze(0).to(DEVICE)
        return clip_model.encode_image(t).cpu().numpy()

def embed_text(text):
    tokenizer = open_clip.get_tokenizer("ViT-B-32")
    with torch.no_grad():
        t = tokenizer([text]).to(DEVICE)
        return clip_model.encode_text(t).cpu().numpy()

# Resolve column names
ASIN_COL = "asin"
FILE_COL = "filename"

# Read data
train = TRAIN.copy()  # already loaded earlier
train[ASIN_COL] = train[ASIN_COL].astype(str)

catalog = pd.read_csv(os.path.join(PROC, "catalog_expanded.csv"))
catalog[ASIN_COL] = catalog[ASIN_COL].astype(str)

K = 5  # use more templates for better coverage
asin_emb = {}

# Image-based centroids
for asin, g in train.groupby(ASIN_COL):
    vecs = []
    for fn in g[FILE_COL].dropna().head(K).tolist():
        path = os.path.join(RAW, fn)
        if os.path.exists(path):
            try:
                vecs.append(embed_img(path))
            except:
                pass
    if vecs:
        asin_emb[asin] = np.vstack(vecs).mean(axis=0, keepdims=True)

# Robust TEXT fallback
text_candidates = [
    c for c in catalog.columns
    if any(k in c.lower() for k in ["title", "name", "desc", "product", "label", "brand"])
]

for asin in train[ASIN_COL].unique():
    if asin not in asin_emb:
        row = catalog[catalog[ASIN_COL] == asin]
        text = None
        if not row.empty:
            for c in text_candidates:
                val = str(row.iloc[0].get(c, "")).strip()
                if val and val.lower() not in ["nan", "none", ""]:
                    text = val
                    break
        if not text:
            text = f"Retail packaged product {asin}"
        try:
            asin_emb[asin] = embed_text(text)
        except:
            asin_emb[asin] = embed_text(f"Product {asin}")

print("✅ asin_emb built")
print("Coverage:", len(asin_emb), "/", train[ASIN_COL].nunique())


/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


✅ asin_emb built
Coverage: 30 / 30


In [ ]:
# CPU-mode: build/save asin_emb here, then use infer.count_asin_in_image
import os, sys, numpy as np, pandas as pd, pickle, torch, open_clip
from PIL import Image
import os, importlib, sys
sys.path.append("/content/drive/MyDrive/abid-mvp/src")

os.environ["CLIP_MODEL"] = "ViT-B-32"
os.environ["CLIP_TAG"]   = "openai"

import infer
importlib.reload(infer)

from infer import count_asin_in_image  # re-import after reload

# --- Paths ---
BASE = "/content/drive/MyDrive/abid-mvp"
RAW  = f"{BASE}/data/raw/bin-images"
PROC = f"{BASE}/data/processed"
MODELS = f"{BASE}/models"
os.makedirs(MODELS, exist_ok=True)

# --- Import the CPU infer (grid crops, ViT-B/32) ---
SRC = f"{BASE}/src"
if SRC not in sys.path: sys.path.insert(0, SRC)
from infer import count_asin_in_image  # only this is needed now

# --- CLIP (ViT-B/32) for building template bank ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, _, preprocess = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")
clip_model = clip_model.to(DEVICE).eval()
tokenizer = open_clip.get_tokenizer("ViT-B-32")

def embed_img(path_or_pil):
    img = Image.open(path_or_pil).convert("RGB") if isinstance(path_or_pil, str) else path_or_pil.convert("RGB")
    with torch.no_grad():
        t = preprocess(img).unsqueeze(0).to(DEVICE)
        return clip_model.encode_image(t).cpu().numpy()

def embed_text(text):
    with torch.no_grad():
        tok = tokenizer([text]).to(DEVICE)
        return clip_model.encode_text(tok).cpu().numpy()

# --- Load dataframes ---
TRAIN = pd.read_csv(f"{PROC}/train.csv")
VAL   = pd.read_csv(f"{PROC}/val.csv")
ASIN_COL = "asin"; FILE_COL = "filename"

# --- Optional catalog for text fallback ---
catalog_path = os.path.join(PROC, "catalog_expanded.csv")
CAT = pd.read_csv(catalog_path) if os.path.exists(catalog_path) else pd.DataFrame()
CAT[ASIN_COL] = CAT.get(ASIN_COL, pd.Series(dtype=str)).astype(str)
TRAIN[ASIN_COL] = TRAIN[ASIN_COL].astype(str)

# pick reasonable text columns for fallback
text_cols = [c for c in CAT.columns if any(k in c.lower() for k in ["title","name","desc","product","label","brand"])]

# --- Build asin_emb (K=5 images per ASIN, else text fallback) ---
K = 5
asin_emb = {}
for asin, g in TRAIN.groupby(ASIN_COL):
    vecs = []
    for fn in g[FILE_COL].dropna().head(K).tolist():
        fp = os.path.join(RAW, fn)
        if os.path.exists(fp):
            try: vecs.append(embed_img(fp))
            except: pass
    if vecs:
        asin_emb[asin] = np.vstack(vecs).mean(axis=0, keepdims=True)

# text fallback for missing
for asin in TRAIN[ASIN_COL].unique():
    if asin in asin_emb:
        continue
    prompt = None
    if not CAT.empty and asin in set(CAT[ASIN_COL]):
        row = CAT.loc[CAT[ASIN_COL]==asin].iloc[0]
        for c in text_cols:
            s = str(row.get(c, "")).strip()
            if s and s.lower() not in ["nan","none","null"]:
                prompt = s; break
    if not prompt:
        prompt = f"Retail packaged product {asin}"
    try:
        asin_emb[asin] = embed_text(prompt)
    except:
        asin_emb[asin] = embed_text(f"Product {asin}")

# --- Save for reuse ---
with open(os.path.join(MODELS, "asin_emb.pkl"), "wb") as f:
    pickle.dump(asin_emb, f)
print(f"✅ asin_emb saved → {MODELS}/asin_emb.pkl | coverage {len(asin_emb)}/{TRAIN[ASIN_COL].nunique()}")

# --- Quick smoke test on a real row from VAL that exists on disk ---
test_row = None
for _, r in VAL.iterrows():
    fp = os.path.join(RAW, r[FILE_COL])
    if os.path.exists(fp) and str(r[ASIN_COL]) in asin_emb:
        test_row = r; break

if test_row is not None:
    fp = os.path.join(RAW, test_row[FILE_COL])
    asin = str(test_row[ASIN_COL])
    cnt, hits, boxes = count_asin_in_image(fp, asin, asin_emb, score_thresh=0.20)  # CPU/grid uses thr≈0.20
    print("🔎 Test:", os.path.basename(fp), "| ASIN:", asin, "| cnt:", cnt, "| top hits:", hits[:3])
else:
    print("⚠️ No valid test row found (file missing or missing embedding).")


/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


✅ asin_emb saved → /content/drive/MyDrive/abid-mvp/models/asin_emb.pkl | coverage 30/30


/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


🔎 Test: 00642.jpg | ASIN: B01BNJL3KC | cnt: 32 | top hits: [(31, 0.9108688235282898), (16, 0.9066426753997803), (9, 0.8938843607902527)]


In [ ]:
def _iou(a, b):
    ax1,ay1,ax2,ay2 = a; bx1,by1,bx2,by2 = b
    inter = max(0, min(ax2,bx2)-max(ax1,bx1)) * max(0, min(ay2,by2)-max(ay1,by1))
    ua = (ax2-ax1)*(ay2-ay1) + (bx2-bx1)*(by2-by1) - inter
    return inter/ua if ua>0 else 0.0

def _nms_hits(hits, boxes, iou_thr=0.5):
    kept = []
    removed = set()
    for i, s in hits:                   # hits sorted by score desc
        if i in removed: continue
        kept.append((i, s))
        for j, _ in hits:
            if j==i or j in removed: continue
            if _iou(boxes[i], boxes[j]) >= iou_thr:
                removed.add(j)
    return kept


In [ ]:
cnt, hits, boxes = count_asin_in_image(fp, asin, asin_emb, score_thresh=0.22)
print("num_crops =", len(boxes))   # expect 7–10 (3×3 minus tiny tiles)
print("count =", cnt)              # should drop from 32 → sane single-digit


num_crops = 9
count = 9


In [ ]:
# how many crops did we even have?
print("num_crops =", len(boxes))  # if using grid_crops, equals grid^2 (minus small tiles)


num_crops = 9


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.8 MB/s eta 0:00:00


In [ ]:
# --- 02_ Embeddings & Detection Preflight ---
import os, sys, pickle, numpy as np, importlib
from PIL import Image

BASE = "/content/drive/MyDrive/abid-mvp"
RAW  = f"{BASE}/data/raw/bin-images"
MODELS = f"{BASE}/models"
SRC  = f"{BASE}/src"
if SRC not in sys.path: sys.path.insert(0, SRC)
import infer; importlib.reload(infer)

# 1) Model & dims
m, _ = infer.load_clip("ViT-B-32", "openai")
d = m.visual.output_dim
print("✅ CLIP:", type(m).__name__, "| dim:", d); assert d==512

# 2) asin_emb exists + normalize (idempotent)
with open(f"{MODELS}/asin_emb.pkl","rb") as f: asin_emb = pickle.load(f)
for k,v in list(asin_emb.items()):
    n = np.linalg.norm(v, axis=1, keepdims=True)
    asin_emb[k] = v / np.clip(n, 1e-9, None)
print("✅ asin_emb:", len(asin_emb), "classes (L2-normalized)")

# 3) YOLO present + fallback grid
assert hasattr(infer, "load_yolo") and hasattr(infer, "grid_crops")
print("✅ Proposals: YOLO-lite + grid fallback wired")

# 4) One real test
import pandas as pd
VAL = pd.read_csv(f"{BASE}/data/processed/val.csv")
row = next(r for _,r in VAL.iterrows() if os.path.exists(os.path.join(RAW, r['filename'])) and str(r['asin']) in asin_emb)
fp  = os.path.join(RAW, row['filename']); asin = str(row['asin'])
cnt, hits, boxes = infer.count_asin_in_image(fp, asin, asin_emb, score_thresh=0.26, imgsz=320, grid=4, min_side=48)
print("✅ Test call OK | crops:", len(boxes), "| count:", cnt, "| top:", (hits[0] if hits else None))

# 5) Crop embedding L2 check (deterministic cosine)
img = Image.open(fp).convert("RGB")
crops,_ = infer.grid_crops(img, grid=4, min_side=48)
if crops:
    E = np.vstack([infer._embed_pil(c) for c in crops])
    E = E / np.clip(np.linalg.norm(E, axis=1, keepdims=True), 1e-9, None)
    print("✅ Crop embeddings L2-normalized:", E.shape)
print("🎯 Preflight passed.")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


✅ CLIP: CLIP | dim: 512
✅ asin_emb: 30 classes (L2-normalized)
✅ Proposals: YOLO-lite + grid fallback wired
✅ Test call OK | crops: 21 | count: 7 | top: (0, 0.9271279573440552)
✅ Crop embeddings L2-normalized: (16, 512)
🎯 Preflight passed.
